In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 7.5 MB/s eta 0:00:00


In [7]:
!nvidia-smi

Thu Mar  7 09:14:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [11]:

# Split the dataset into train and test and creates the train.txt and test.tx with
# the respective path of the images in each folder


def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue

      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

train_test_split('/content/drive/MyDrive/yolov8/data/')


------ PROCESS STARTED -------
--- This folder has a total number of 355 images---


  0%|          | 0/284 [00:00<?, ?it/s]

------ Training data created with 80% split 284 images -------


  0%|          | 0/71 [00:00<?, ?it/s]

------ Testing data created with a total of 71 images ----------
------ TASK COMPLETED -------


In [12]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/78.2 GB disk)


In [13]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=20 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=football


100% 49.7M/49.7M [00:00<00:00, 65.0MB/s]
Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=football2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

image 1/9 /content/drive/MyDrive/yolov8/test_images/1-1667-_jpg.rf.ebed770cd030f6c693f5a553d69d486e.jpg: 640x640 4 footballs, 36.3ms
image 2/9 /content/drive/MyDrive/yolov8/test_images/1-299-_jpg.rf.eeb0d3ea3b4298c1bc87e738e4a4d7be.jpg: 640x640 (no detections), 36.3ms
image 3/9 /content/drive/MyDrive/yolov8/test_images/1-393-_jpg.rf.6a051d6b1d5ce662b9441252e10a557d.jpg: 640x640 1 football, 36.3ms
image 4/9 /content/drive/MyDrive/yolov8/test_images/1-946-_jpg.rf.ab446ee2c398cd20b886779ef2219b3c.jpg: 640x640 1 football, 36.3ms
image 5/9 /content/drive/MyDrive/yolov8/test_images/PXL_20230613_123302564.jpg: 640x384 (no detections), 75.1ms
image 6/9 /content/drive/MyDrive/yolov8/test_images/PXL_20230728_104035644.jpg: 384x640 1 football, 70.3ms
image 7/9 /content/drive/MyDrive/yolov8/test_images/football-8001067_1280.jpg: 44

In [15]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [20]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt conf=0.35 source=/content/drive/MyDrive/yolov8/vid1


Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

video 1/2 (1/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 191.6ms
video 1/2 (2/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 24.7ms
video 1/2 (3/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 25.1ms
video 1/2 (4/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 24.7ms
video 1/2 (5/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 25.8ms
video 1/2 (6/734) /content/drive/MyDrive/yolov8/vid1/_import_616d52a9f1ff22.16208340_720p_5000br.mp4: 384x640 (no detections), 24.6ms
video 1/2 (7/734) /content

In [21]:
!cp -r /content/runs/detect/predict4 /content/drive/MyDrive/yolov8/output

In [22]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt format=onnx

Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (49.6 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 2.8s, saved as '/content/drive/MyDrive/yolov8/training_results/football2/weights/best.onnx' (98.8 MB)

Export complete (7.7s)
Results saved to /content/drive/MyDrive/yolov8/training_results/football2/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.onnx imgsz=640 data=/content/drive/MyDrive/yolov8/dataset.yaml  
Visualize:       https://netron.app
💡 Learn more at https